<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://erachelson.github.io/RLclass_MVA/">https://erachelson.github.io.github.io/RLclass_MVA/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Chapter 1: Modeling sequential decision problems with Markov Decision Processes</div>

<div class="alert alert-success">

**Learning outcomes:**  
This chapter is mostly about definitions. By the end of this chapter you should be able to define and discuss what is:
- a Markov decision process,
- an action policy,
- a value function,
- an optimal policy / value function.

Additionally, after doing the homework, you should be able to define and discuss:
- Monte Carlo estimation of values,
- the stationary distribution of a Markov decision process,
- the state occupancy measure of a policy given a starting distribution,
- the horizon under a $\gamma$-discounted criterion.

You should also be familiar with the Gymnasium API.
</div>

# Definition

Let's take a higher view and develop a general theory for describing problems such as writing a prescription for the patient we considered in the previous class.

Let us assume we have:
- a set of states $S$ describing the system to control,
- a set of actions $A$ we can apply.

Curing patients is a conceptually difficult task. 
To keep things grounded, we shall use a toy example called [FrozenLake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) and work our way to more general concepts. It's also the occasion to familiarize with [Gymnasium](https://gymnasium.farama.org/).

<center><img src="img/frozenlake.jpg" style="height: 300px;"></img></center>

In [ ]:
import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl
# use render_mode="human" to open the game window
env = gym.make('FrozenLake-v1', render_mode="ansi")
env.reset()
print(env.render())

In [ ]:
## Run this only if you have used render_mode="human" in the cell above
#env.close()

The game's goal is to navigate across this lake, from position S to position G, while avoiding falling into the holes H. Frozen positions are slippery so you don't always move in the intended direction. Reaching the goal provides a reward of 1, and zero otherwise. Falling into a hole or reaching the goal ends an episode.

A more complete description of the environment is provided in [Gymnasium's documentation](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) (or through `help(fl.FrozenLakeEnv)`).

<div class="alert alert-warning">
    
**Poll:**  
[https://linkto.run/p/65E9EO4Q](https://linkto.run/p/65E9EO4Q)  
How many states are there in this game?  
How many actions?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

States set: the 16 positions on the map.  
Actions set: the 4 actions $\{$N,S,E,W$\}$.
</details>

Let's confirm that:

In [ ]:
print(env.observation_space)
print(env.action_space)

At every time step, the system state is $S_t$ and we decide to apply action $A_t$. This results in observing a new state $S_{t+1}$ and receiving a scalar reward signal $R_t$ for this transition.

$R_t$ tells us how happy we are with the last transition.

For example, in FrozenLake, all transitions have reward 0 except for the one that reaches the goal, which yields reward 1. Let's verify this in the code and introduce a few utility functions on the way.

Note that $S_t$, $A_t$, $S_{t+1}$ and $R_t$ are random variables.

In [ ]:
# %load solutions/RL1_utility_functions.py
import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl

actions = {fl.LEFT: '\u2190', fl.DOWN: '\u2193', fl.RIGHT: '\u2192', fl.UP: '\u2191'}

def to_s(row,col):
    env = gym.make('FrozenLake-v1', render_mode="ansi")
    return row*env.unwrapped.ncol+col

def to_row_col(s):
    env = gym.make('FrozenLake-v1', render_mode="ansi")
    col = s%env.unwrapped.ncol
    row = int((s-col)/env.unwrapped.ncol)
    return row,col


In [ ]:
import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl
env = gym.make('FrozenLake-v1', render_mode="ansi")

print(actions)
row=3
col=2
a=2
print("Apply ", actions[2], " from (", row, ", ", col, "):", sep='')
for tr in env.unwrapped.P[to_s(row,col)][a]:
    print("  Reach (", to_row_col(tr[1]), ") and get reward ", tr[2], " with proba ", tr[0], ".", sep='')

In [ ]:
env.unwrapped.P[to_s(row,col)][a]

We will now make our main assumption about the systems we want to control.

<div class="alert alert-success">
    
**Fundamental assumption (Markov property)**
$$\mathbb{P}(S_{t+1},R_t|S_t, A_t, S_{t-1}, A_{t-1}, \ldots, S_0, A_0) = \mathbb{P}(S_{t+1},R_t|S_t, A_t)$$
</div>
    
Such a system will be called a Markov Decision Process (MDP).

One generally separates the state dynamics and the rewards by:
$$\mathbb{P}(S_{t+1},R_t|S_t, A_t) = \mathbb{P}(S_{t+1}|S_t, A_t)\cdot \mathbb{P}(R_t|S_t, A_t, S_{t+1})$$

Which leads in turn to the general definition of an MDP:
<div class="alert alert-success"><b>Markov Decision Process (MDP)</b><br>
A Markov Decision Process is given by:
<ul>
<li> A set of states $S$
<li> A set of actions $A$
<li> A (Markovian) transition model $\mathbb{P}\left(S_{t+1} | S_t, A_t \right)$, noted $p(s'|s,a)$
<li> A reward model $\mathbb{P}\left( R_t | S_t, A_t, S_{t+1} \right)$, noted $r(s,a)$ or $r(s,a,s')$
<li> A set of discrete decision epochs $T=\{0,1,\ldots,H\}$
</ul>
</div>

Most of the results presented here can be found in M. L. Puterman's classic book, **[Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://www.wiley.com/en-us/Markov+Decision+Processes%3A+Discrete+Stochastic+Dynamic+Programming-p-9781118625873)**.

If $H\rightarrow\infty$ we have an infinite horizon control problem.

<div class="alert alert-success">

Since we will work most of the time with infinite horizon problems, we shall identify the MDP with the 4-tuple $\langle S,A,p,r\rangle$.    
</div>


In the general case, $S$ and $A$ may each be either:
  - arbitrary finite sets,
  - arbitrary countable infinite sets,
  - compact subsets of a finite dimensional Euclidean space, or
  - non-empty Borel subsets of complete, separable metric spaces.

The reward model can be written interchangeably $r(s,a)$ or $r(s,a,s')$, depending on authors, with $r(s,a) = \mathbb{E}_{s'\sim p(\cdot|s,a)} [r(s,a,s')]$. We will use both notations in the rest of the class.

So, in RL, we wish to control the trajectory of a system that, we suppose, behaves as a Markov Decision Process.

<center><img src="img/dynamic.png" style="height: 240px;"></img></center>

# Policies

Formally, how does one write the behavior of an agent?
This behavior specifies how to choose actions at each time step:
$$A_t \sim \pi_t.$$

Let $\Delta_A$ be the set of probability measures on the action space $A$. Then the law $\pi_t$ of $A_t$ belongs to $\Delta_A$. 
$\pi_t$ is called the **decision rule** at step $t$, it is a distribution over the action space $A$.  
The collection $\pi = \left(\pi_t \right)_{t\in T}$ is the oracle's **policy**.

<div class="alert alert-success"><b>Policy $\pi$</b><br>
A policy $\pi$ is a sequence of decision rules $\pi_t$: $\pi = \{\pi_t\}_{t\in T}$, with $\pi_t \in \Delta_A$.
</div>

One policy implies one specific distribution over trajectories over the frozen lake. More generally, the policy and $S_0$ condition the sequence $S_0, A_0, R_0, S_1, A_1, R_1, \ldots$

# Value of a trajectory / of a policy

In FrozenLake as in the patient's example, some trajectories are better than others. We need a criterion to compare trajectories. Intuitively, this criterion should reflect the idea that a good policy accumulates as much reward as possible along a trajectory.

Let's compare the policy that always moves to the right and the policy that always moves left by summing the rewards obtained along trajectories and then averaging these rewards across trajectories.

In [ ]:
import numpy as np
nb_episodes = 50000
horizon = 200

Vright = np.zeros(nb_episodes)
for i in range(nb_episodes):
    env.reset()
    for t in range(horizon):
        next_state, r, done, trunc, _ = env.step(fl.RIGHT)
        Vright[i] += r
        if done:
            break

Vleft  = np.zeros(nb_episodes)
for i in range(nb_episodes):
    env.reset()
    for t in range(horizon):
        next_state, r, done, trunc, _ = env.step(fl.LEFT)
        Vleft[i] += r
        if done:
            break

print("est. value of 'right' policy:", np.mean(Vright), "std dev:", np.std(Vright))
print("est. value of 'left'  policy:", np.mean(Vleft),  "std dev:", np.std(Vleft))

In the general case, this sum of rewards on an infinite horizon might be unbounded. So let us introduce the **$\gamma$-discounted sum of rewards** (from a starting state $s$, under policy $\pi$) random variable:
$$G^\pi(s) = \sum\limits_{t = 0}^\infty \gamma^t R_t \quad \Bigg| \quad \begin{array}{l}S_0 = s,\\ A_t \sim \pi_t,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$

$G^\pi(s)$ represents what we can gain in the long-term by applying the actions from $\pi$.

Then, given a starting state $s$, we can define the value of $s$ under policy $\pi$:
$$V^\pi(s) = \mathbb{E} \left[ G^\pi(s) \right]$$

This defines the value function $V^\pi$ of policy $\pi$:
<div class="alert alert-success"><b>Value function $V^\pi$ of a policy $\pi$ under a $\gamma$-discounted criterion</b><br>
$$V^\pi : \left\{\begin{array}{ccl}
S & \rightarrow & \mathbb{R}\\
s & \mapsto & V^\pi(s)=\mathbb{E}\left( \sum\limits_{t = 0}^\infty \gamma^t R_t \bigg| S_0 = s, \pi \right)\end{array}\right. $$
</div>

And, given a distribution $\rho_0$ on starting states, we can map $\pi$ to the scalar value:
$$J(\pi) = \mathbb{E}_{s \sim \rho_0} \left[ V^\pi(s) \right]$$

Note that this definition is quite arbitrary: instead of the expected (discounted) sum of rewards, we could have taken the average reward over all time steps, or some other (more or less exotic) comparison criterion between policies. For example:
<table>
<tr>
    <td>Finite horizon</td>
    <td>$V(s) = \mathbb{E}\left( \sum\limits_{t = 0}^H R_t \bigg| s_0 = s \right)$ </td>
    <td width="150px">with $H \in \mathbb{N}$</td>
</tr>
<tr>
    <td> Average reward </td>
    <td width="300px">$V(s) = \mathbb{E}\left( \lim\limits_{H\rightarrow\infty}  \frac{1}{H} \sum\limits_{t = 0}^H R_t \bigg| s_0 = s \right)$ </td>
    <td></td>
</tr>
<tr>
    <td> Total reward </td>
    <td>$V(s) = \mathbb{E}\left( \lim\limits_{H\rightarrow\infty} \sum\limits_{t = 0}^H R_t \bigg| s_0 = s \right)$ </td>
    <td></td>
</tr>
<tr>
    <td>Discounted reward </td>
    <td>$V(s) = \mathbb{E}\left( \lim\limits_{H\rightarrow\infty} \sum\limits_{t = 0}^H \gamma^t R_t \bigg| s_0 = s \right)$ </td>
    <td width="150px">with $0\leq \gamma<1$</td>
</tr>
</table>

- The average reward criterion characterizes the average reward per time step the agent gets. This can be useful in some control applications. However, in the case of FrozenLake, we don't want to average our rewards, we want to get to the goal as soon as possible.
- The total reward criterion seems more adapted: it maximizes the cumulated rewards obtained during an episode. But it does not discriminate whether they were obtained at the beginning or late in the episode. Additionally, it suffers from a major flaw: for infinite horizon problems, even if the reward model is bounded, this sum might diverge. So we need a better formulation for the general case of infinite horizon problems.
- The discounted reward criterion suits our needs. The discount factor ($0\leq \gamma<1$) guarantees that with bounded reward models $r$, the sum always converges. Also it has the properties we desire: a reward of 1 obtained at the first time step weights 1 in the final criterion, while a reward of 1 obtained after $t$ time steps only weights $\gamma^t$; it is *discounted* by $\gamma^t$ (hence the criterion's name).

Most of the RL literature uses this discounted criterion (in some cases with $\gamma=1$), some works use a finite horizon criterion, some use the average reward criterion, and few works venture into more exotic criteria. For now, we will limit ourselves to the discounted criterion.

# Optimal policies

The fog clears up a bit: we can now compare policies given an initial state (or initial state distribution).  

Thus, an **optimal** policy is one that is better than any other.

<div class="alert alert-success"><b>Optimal policy $\pi^*$</b><br>
$\pi^*$ is said to be optimal iff $\pi^* \in \arg\max\limits_{\pi} V^\pi$.<br>
<br>
    
A policy is optimal if it **dominates** over any other policy in every state:
$$\pi^* \textrm{ is optimal}\Leftrightarrow \forall s\in S, \ \forall \pi, \ V^{\pi^*}(s) \geq V^\pi(s)$$
</div>

Note that although there may be several optimal policies, they all share the same value function $V^* = V^{\pi^*}$.

We now get to our first fundamental result. Fortunately for us...  

<div class="alert alert-success"><b>Theorem: family of optimal policies</b><br>
For $\left\{\begin{array}{l}
\gamma\textrm{-discounted criterion}\\
\textrm{infinite horizon}
\end{array}\right.$, 
there always exists at least one optimal stationary, deterministic, memoryless policy.
</div>

Let's explain a little:
- Memoryless (also called Markovian): all decision rules are only conditioned by the last seen state. Mathematically: 
$\left.\begin{array}{l}
\forall \left(s_i,a_i\right)_{i\leq t-1}\in \left(S\times A\right)^{t-1}\\
\forall \left(s'_i,a'_i\right)_{i\leq t-1}\in \left(S\times A\right)^{t-1}\\
\forall s \in S
\end{array}\right\}, \pi_t\left(A_t|S_0=s_0, A_0=a_0, \ldots, S_t=s\right) = \pi_t\left(A_t|S'_0=s'_0, A'_0=a'_0, \ldots, S_t=s\right)$.  
One writes $\pi_t(A_t|S_t=s)$, or more simply $\pi_t(\cdot | s)$ or $\pi_t(s) \in \Delta_A$.
- Stationary (and memoryless): all decision rules are the same throughout time. Mathematically:  
$\forall (t,t')\in \mathbb{N}^2, \pi_t(A_t|S_t=s) = \pi_t(A_{t'}|S_{t'}=s)$.  
This unique distribution is written $\pi(\cdot | s) = \pi_t( \cdot | s)$ or $\pi(s) \in \Delta_A$.
- Deterministic: all decision rules put all probability mass on a single item of the action space $A$.  
$\pi_t(A_t|history) = \left\{\begin{array}{l}
1\textrm{ for a single }a\\
0\textrm{ otherwise}
\end{array}\right.$.

So in simpler words, we know that among all possible optimal ways of picking $A_t$, at least one is a function $\pi:S\rightarrow A$.

That helps a lot: we don't have to search for optimal policies in a complex family of history-dependent, stochastic, non-stationary policies; instead we can simply search for a function $\pi(s)=a$ that maps states to actions.

<details class="alert alert-danger">
    <summary markdown="span"><b>Proof sketch (click to expand)</b></summary>

The proof (very simple but a little long) is in chapter 6 of the <b>Markov Decision Processes</b> book by Martin L. Puterman.<br>
To give you the general flavour:
<ul>
<li> The infinite horizon leads to the existence of an optimal <b>stationary</b> policy: if the horizon is infinitely far, the optimal decision rule $n$ steps before the end if the same as the one $n+1$ steps before the end (watch out, this intuition can be very false in other contexts).
<li> The <b>Markovian</b> property of $p(s'|s,a)$ allows to get optimal memoryless policies.
<li> The <b>deterministic</b> part is somehow more tricky but just note that this result only holds for single-player MDPs. For a two-agents competitive game for example (like poker for instance), there is no deterministic optimal policy.
</ul>
</details>

Remark: quite often, we will retain the memoryless and stationary properties of optimal policies but still search for stochastic ones. Why we do this is still a bit beyond the current notebook (it involves notions of exploration, sample distribution, and smoothness of the optimization landscape). For now, let us just remember we shall search for optimal policies under the form of functions from $S$ to either $A$ (deterministic policies, $\pi(s)\in A$) or $\Delta_A$ (stochastic policies, $\pi(s) \in \Delta_A$).

# A weaker notion of optimality 

If all states have a non-zero probability of being visited by any policy, then an optimal policy should pick optimal actions along any trajectory starting in (any) $s_0$, so it should pick optimal actions in all states. Then finding a policy which maximize $V^\pi$ in all states is actually the same as finding a policy which maximizes the expected value $V^\pi(s_0)$ of a fixed initial state $s_0$, or the expected value over any distribution on initial states $\mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$.

To fix ideas, let's write $J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$. Then:
<div class="alert alert-success">

**The policy optimization problem:**  
Provided all states are reachable from any other state, an optimal policy is a solution to $\max_\pi J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$.
</div>

So we turned a problem of value maximization in *every state* into one of value maximization *on average* across a distribution $\rho_0$. Note that this distribution need not be a specific initial state distribution.

The assumption that all states are reachable from any other one is very strong. When it is not verified (which will happen in many real-life cases), the optimization problem $\max_\pi J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$ might not yield a fully optimal policy. It will provide a policy which maximizes its expected outcome on average across states, on a specific set of starting states distributed according to $\rho_0$.

This provides us with a weaker definition of optimality, stating that we search for a policy which maximizes its expected gain, *on average across $\rho_0$*.

# A cartography of optimization methods

The two notions of optimality introduced above pave the way to two different families of methods in RL. Either one searches for the optimal value function (strong optimality), and then derives an optimal policy as a by-product, or one directly optimizes for good average-value policies (weak optimality). In practice, no approach is more justified than the other and both deserve studying. This enables separating families of methods and drawing a map of future classes.

Policy optimization, solve the $\max_\pi$ problem directly:
- by derivative-free methods (evolutionary RL)
- by derivative-based methods (policy gradient methods and their modern extensions)

Value optimization, solve for $V^*$:
- by (approximate) dynamic programming (and the vast span of most recent approximate value iteration algorithms)
- by linear programming or alternate formulations

# Limits of the MDP model

What if the system is an MDP but its state is not fully observable?  
$\rightarrow$ This is the (exciting) field of Partially Observable MDPs. Our key result of having a Markovian optimal policy does not hold anymore. There are ways to still obtain optimal policies (but it is often very computationaly costly) or approximate them with Markovian policies.

What happens if there are multiple actions taken at the same time by different agents?  
$\rightarrow$ This falls into the category of multi-player stochastic games. Such games can be adversarial, cooperative, or a mix of the two. Of course they can also have partial observability.

What if the transition model is not Markovian?  
$\rightarrow$ Beware, here be dragons! All the beautiful framework above crumbles down if its hypothesis are violated. So great care should be taken when choosing the state variables for a given problem. In a sense, an MDP is a discrete time version of a first-order differential equation. Writing a system as $\dot{X} = f(X,U, noise)$ as is common in control theory is a good practice to ensure the Markov property.

# Summary

Let's wrap this section up. Our goal was to formally define the search for the best strategy for our game of FrozenLake and the medical prescription problem. This has led us to formalizing the general **discrete-time stochastic optimal control problem**:
- Environment (discrete time, non-deterministic, non-linear, Markov) $\leftrightarrow$ MDP.
- Behaviour $\leftrightarrow$ control policy $\pi : S\rightarrow A$ or $\Delta_A$.
- Policy evaluation criterion $\leftrightarrow$ $\gamma$-discounted criterion.
- Goal $\leftrightarrow$ Maximize value function $V^\pi(s)$.

So we have built the first stage of our three-stage rocket:  
<div class="alert alert-success">
    
**What is the system to control?**  
The system to control is a Markov Decision Process $\langle S, A, p, r \rangle$ and we will control it with a policy $\pi:s\mapsto a$ in order to optimize $\mathbb{E} \left( \sum_t \gamma^t R_t\right)$
</div>

We can now move on to the next question: how does one find an optimal strategy?

<div class="alert alert-warning">

**Poll** The limits of MDP modeling  
[https://linkto.run/p/0WG7WNER](https://linkto.run/p/0WG7WNER)  
Can these systems be modeled as MDPs?   
- Playing a tennis video game based on a single video frame
- Playing a tennis video game based on a full physical description of the ball and the players
- The game of Poker
- The collaborative game of [Hanabi](https://en.wikipedia.org/wiki/Hanabi_(card_game))
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

A single video frame does not contain enough information to accurately represent the current state of the game. The velocities are absent for instance. Hence the dynamics might not be Markovian.
    
A full physical description, however, may contain enough information so that $\mathbb{P}(S_{t+1})$ is only conditioned by $S_t$ and $A_t$.
    
Poker is a two-player, adversarial, stochastic game. MDPs only model one-player games.

Beyond the fact that it is a multi-player game. Hanabi is a game based mainly on epistemic reasoning. That is, reasoning on beliefs about the state of the world (specifically, the state of the other players' hand). This type of state description is difficult to encode within a Markovian dynamics model.
</details>

# Homework

The exercises below are here to help you play with the concepts introduced above, to better grasp them. They also introduce additional important notions. They are not optional to reach the class goals. Often, the provided answer reaches out further than the plain question asked and provides comments, additional insights, or external references.

## Policies

<div class="alert alert-warning">
    
**Exercise**  
In the text above, we wrote that $\pi_t$ is the distribution over the action space $A$ for the action $A_t$ taken at time step $t$.  
- Write this probability $\mathbb{P}(A_t)$ as a conditional probability $\pi_t(A_t|\ldots)$ (the real question is: what are the "$\ldots$"?).
- Rephrase, with your own words, what this $\pi_t(A_t|\ldots)$ indicates.  
Then we defined a policy $\pi$ as the collection of decision rules $\left( \pi_t \right)_{t\in\mathbb{N}}$.
- Using the answer to the previous questions, write the definition of a Markovian policy, then a stationary Markovian policy (the answer is actually in the text just after the Optimal policy theorem, the exercise is about being able to recall and explain the definitions and what they imply). 
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

- $\pi_t$ describes the distribution over actions at time step $t$. Because of causality (future events don't affect current events), it can only depend on the realization of the state and actions random variables in previous time steps:
$$\mathbb{P}(A_t) = \pi_t(A_t | S_0, A_0, \ldots, S_{t-1}, A_{t-1}, S_t)$$
We define the *history* $H_t = S_0, A_0, \ldots, S_{t-1}, A_{t-1}, S_t$ at time step $t$ as this random sequence. So:
$$\mathbb{P}(A_t) = \pi_t(A_t | H_t)$$

- In plain words, for an action $a$ and a history $h$ at step $t$, $\pi_t(a|h)$ indicates  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; the probability to pick action $a$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; at time $t$,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; given the history of states/actions $h$.  
This is called a *history-dependent, non-stationary, stochastic* policy and is the most generic class of policies.

- In a Markovian policy, all decision rules are only conditioned by the last encountered state.
$$\pi_t(A_t|H_t) = \pi_t(A_t | S_t)$$
In other words, given two (possibly different) sequences of state-action random variables realizations up to time $t-1$ and a single realization of S_t the distribution of $A_t$ is the same.
Mathematically: 
$\left\{\begin{array}{l}
\forall \left(s_i,a_i\right)_{i\leq t-1}\in \left(S\times A\right)^{t-1}\\
\forall \left(s'_i,a'_i\right)_{i\leq t-1}\in \left(S\times A\right)^{t-1}\\
\forall s \in S
\end{array}\right.,$
\begin{align*}
    \pi_t(A_t|H_t) &= \pi_t\left(A_t|S_0=s_0, A_0=a_0, \ldots, S_t=s\right)\\
    &= \pi_t\left(A_t|S'_0=s'_0, A'_0=a'_0, \ldots, S_t=s\right)\\
    &= \pi_t(A_t | S_t)
\end{align*}
One writes $\pi_t(A_t|S_t=s)$, or more simply $\pi_t(\cdot | s)$.  
In a stationary Markovian policy, all decision rules are the same throughout time. Mathematically:  
$\forall (t,t')\in \mathbb{N}^2, \pi_t(A_t|S_t=s) = \pi_t(A_{t'}|S_{t'}=s)$.  
This unique distribution is written $\pi(\cdot | s) = \pi_t( \cdot | s)$.
</details>

<div class="alert alert-warning">

**Exercise**  
In the patient example, suppose the physician tells the patient to take drug A every day for 5 days, then drug B every two days for 9 days, then come back for a check-up. The physician adds to take drug C once a day if the patient feels pain over two consecutive days. Can you write the sequence of corresponding decision rules?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

This prescription is made over a finite horizon $H=14$ days. The actions are the combinations of drugs $A=\left\{ \emptyset, (A), (B), (C), (A,B), (A,C), (B,C), (A,B,C) \right\}$.   
    
The prescription is deterministic: the distribution over actions is a Dirac. We will write it $a_t = \pi_t(h)$.
    
The prescription depends on the two last states of the patient. So it's not Markovian, it is history-dependent. Precisely, it depends on the boolean state variable "is there pain?". So we can write $\pi_t(h) = \pi_t(s_t,s_{t-1})$.  
  
It also is not stationary, since the prescription changes after day 5.  
    
Consequently, the policy is:  
For $t \in [1, 5]$:   
if $pain(s_t,s_{t-1})=True$, $\pi_t(s_t,s_{t-1}) = (A,C)$,  
if $pain(s_t,s_{t-1})=False$, $\pi_t(s_t,s_{t-1}) = (A)$.  
For $t \in [6, 14]$:   
if $t$ is even and $pain(s_t,s_{t-1})=True$, $\pi_t(s_t,s_{t-1}) = (B,C)$,  
if $t$ is even and $pain(s_t,s_{t-1})=False$,  $\pi_t(s_t,s_{t-1}) = (B)$,  
if $t$ is odd and $pain(s_t,s_{t-1})=True$, $\pi_t(s_t,s_{t-1}) = (C)$,  
if $t$ is odd and $pain(s_t,s_{t-1})=False$,  $\pi_t(s_t,s_{t-1}) = \emptyset$
</details>

## Monte Carlo value estimation

<div class="alert alert-warning">

**Exercise**  
Write a `mc_eval(pi,nb_trials)` function that uses the FrozenLake environment we've introduced earlier to obtain a vector of length `nb_trials` containing the return realizations for `nb_trials` Monte-Carlo rollouts of policy `pi` (given as an array of action indices) starting in the initial state $s_0$. Take $\gamma = 0.9$. Yes, the code is almost the same as the example provided earlier.  
Run this function with the policy that always goes right and for 100000 trials.  
Note that $\gamma^{200} \approx 10^{-9}$ so any reward obtained after 200 time steps will have a negligible contribution to $V^\pi(s_0)$, thus rolling an episode out for 200 time steps should be sufficient.
</div>

In [ ]:
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line in the cell below to load a correction (then you can execute this code).

In [ ]:
# %load solutions/RL1_exercise1.py

In [ ]:
import gymnasium as gym
import gymnasium.envs.toy_text.frozen_lake as fl
import numpy as np

env = gym.make('FrozenLake-v1', render_mode="ansi")
pi = fl.RIGHT*np.ones((env.observation_space.n))
Vepisode = mc_eval(pi,100000)
print("value estimate:", np.mean(Vepisode))
print("return variance:", np.std(Vepisode))

## Markov strikes back: transition kernels and stationary distribution

In this section, we get back to basics on Markov chains and play a bit with core notions to acquire a better understanding of MDP properties.

Let's consider a given finite state space MDP and a fixed policy. To set ideas, take the FrozenLake environment and the deterministic policy that always moves right. Let's initialize the MDP to a starting state $s_0$ drawn from a distribution $\rho_0(s)$ and let's look at how the state evolves across time steps. 

<div class="alert alert-warning">

**Exercise**  
Use the MDP's transition model and the policy to define the transition probability $p^\pi(s'|s)$ of reaching $s'$ from $s$.  
What is $\mathbb{P}(s_{t+1})$ given $p^\pi$ and $\mathbb{P}(s_t)$?  
What is the state probability $\mathbb{P}(s_{t+k})$ after $k$ transitions, given $p^\pi$ and $\mathbb{P}(s_t)$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

For a deterministic policy, 
$$p^\pi(s'|s) = p(s'|s,\pi(s)).$$ 

For a stochastic policy, 
$$p^\pi(s'|s) = \mathbb{E}_{a\sim \pi(\cdot | s)} [ p(s'|s,a) ] = \sum_{a\in A} p(s'|s,a) \pi(a|s).$$
    
$\mathbb{P}(s_{t+1}) = \mathbb{P}(s_t) \cdot p^\pi$ (because the MDP equipped with the policy is a Markov chain)

$\mathbb{P}(s_{t+k}) = \mathbb{P}(s_t) \cdot (p^\pi)^k$ ($k$-step transition matrix in a Markov chain).
</details>

The stochastic process of $S_t$ is a Markov chain (as we saw in the previous exercise, since $\pi$ is fixed, the probability of reaching $S_{t+1}$ is only conditionned by $S_t$). 
Let's adopt the convention that $\mathbb{P}(S)$ is a row vector (for a certain ordering of states in $S$), and let's note $p^\pi$ the $|S|\times |S|$ stochastic matrix where $p^\pi_{ss'} = p^\pi(s'|s)$. 
Note that the first line of $p^\pi$ is the distribution over $s_{t+1}$ provided that $s_t$ is the first state. 
Let's use our notations and recall a few results about Markov chains:
- State $s'$ is said to be *reachable* from state $s$ is there exists a sequence of transitions originating in $s$ and ending in $s'$ with positive probability.
- The Markov chain is *irreducible* if all states are reachable from any other state.
- The *period* of a state $s$ is the greatest common divisor of the number of steps required to reach $s$ from itself.
- Two states are said to *communicate* with each other if both are reachable from one another. 
- Two communicating states have the same period.
- A *class* of states is a set of communicating states.
- An irreducible Markov chain has a single class.
- An irreducible Markov chain is *aperiodic* if all states have period one.
- A class $C'$ is *accessible* from another class $C$ if there exists $(s,s')\in C\times C'$ such that $s'$ is accessible from $s$.
- A class is *closed (or terminal, or absorbing)* if the probability of leaving the class is zero. Otherwise it is called *transient*.
- A state is *positive recurrent* if one is guaranteed to come back to it in finite time.
- A state that is positive recurrent and aperiodic is called *ergodic*.
- If all states in an irreducible Markov chain are ergodic, then the chain is said to be ergodic.
- Equivalently, a Markov chain is ergodic if any state can be reached from any other state in a bounded number of steps.
- A *stationary distribution* is a distribution $\rho$ such that $\rho = \rho \cdot p^\pi$.
- If there is at least one positive recurrent state $s$ in the chain, then there exists at least one stationary distribution $\rho$ and $\rho(s)>0$.
- Conversely, if there exists a stationary distribution $\rho$ (such that $\rho = \rho \cdot p^\pi$) and $\rho(s)>0$, then $s$ is positive recurrent.
- If there is a *single* final class, then there is at most one stationary distribution.
- If the Markov chain is irreductible and aperiodic (furthermore if it is ergodic), when $k\rightarrow\infty$, $(p^\pi)^k$ tends to a matrix whose lines are all equal to the stationary distribution of the Markov chain.
- Consequently, if the Markov chain is irreducible and aperiodic, in the long run, the distribution of states follows a stationary distribution $\rho^\pi(s|s_0) = \lim_{k\rightarrow \infty} \rho_0 (p^\pi)^k$, for all possible initial state distribution $\rho_0$. 

A few examples to make this concrete:
- A discretized inverted pendulum with a random action policy is an irreducible and aperiodic Markov chain (but not necessarily ergodic as some states might be asymptotically long to reach).  
- The FrozenLake MDP, under any policy, is never an irreducible Markov chain because of the absorbing "hole in the ice" states.
- The holes in the ice of the FrozenLake MDP, under any policy, are (each) a set of final classes.  
- Suppose we abstract the holes in the ice as a single absorbing state, then under any policy, this state forms a unique final class and the stationary distribution concentrates all probability mass on it.

<div class="alert alert-warning">
    
**Exercise (open discussion):**  
Depending on the policy used, the Markov chain will have different properties. 
In particular, the stationary distribution is not necessarily unique: it depends on $s_0$. 
Here are a few informal examples for you to practice. Can you list a few more?  
What about an Atari 2600 video game like Pong, under an optimal policy?  
A self-driving car under a "reasonably safe" policy?  
The patient example with a chronic disease, under a policy that fights off the disease?   
The patient with a deadly disease under a policy that doesn't cure them?  
The Mad Hatter's casino (from a previous class) under a fixed random policy?  
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

The patient with a chronic disease under a policy that fights off the disease will most likely live a rather long life (let's say infinite, for the sake of this example) and will explore states that are linked to the evolution of the disease. The states corresponding to non-recoverable situations however will not be visited. The Markov chain is not irreducible: depending on the initial state (recoverable situation or not) there are at least two subgraphs describing it. The same goes for the car driven by a cautious driver: many states are visited but not the ones that cause an accident.
    
The patient with a deadly disease and a bad treatment policy will likely die, sadly. On an infinite horizon, the stationary distribution only has probability mass on the states corresponding to death, which is the single absorbing class.
    
Similarly, the FrozenLake example (or the Pong game) has several terminal states, either by reaching the goal or by falling into a hole. It should be noted however that for such episodic environments, it is possible to define an alternate distribution $\rho^\pi(s|s_0)$ that describes the distribution of states before termination.
    
Finally, the Mad Hatter's casino under a fixed random policy is a very nice ergodic Markov chain: from any starting state there is a non-zero probability of reaching any state in a finite number of steps. No terminal states in wonderland!
</details>

## Summing rewards over states rather than time steps: the state occupancy measure

Recall that we introduced $J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$. This is the value of policy $\pi$, evaluated on average across starting states distributed according to $\rho_0$. So:
$$J(\pi) = \mathbb{E}_{(R_t)_{t\in\mathbb{N}}} \left[ \sum\limits_{t = 0}^\infty \gamma^t R_t \bigg| \rho_0,\pi \right]$$

For the sake of simplicity, let's write:
$$r^\pi(s) = \mathbb{E}_{\substack{a\sim \pi \\ s'\sim p^\pi}} [r(s,a,s')].$$

So we can write:
$$J(\pi) = \mathbb{E}_{(S_t)_{t\in\mathbb{N}}} \left( \sum\limits_{t = 0}^\infty \gamma^t r^\pi(S_t) \bigg| \rho_0, \pi \right).$$

We can remark each of the terms under the sum is concerned with a single time step, and its value depends only on $S_t$. 
We can swap the expectation of the sum (over time steps) by the sum (over times steps) of expectations:
$$J(\pi) = \sum\limits_{t = 0}^\infty \gamma^t \mathbb{E}_{S_t} \left( r^\pi(S_t) | \rho_0, \pi \right). $$

Let's write $p(S_t=s|\rho_0,\pi)$ the probability density of encountering each state $s$ at time step $t$, provided that the starting state was drawn according to $\rho_0$ and that the MDP evolves under policy $\pi$. Then:
$$J(\pi) = \sum\limits_{t = 0}^\infty \gamma^t \int_S r^\pi(s) p(S_t=s|\rho_0,\pi) ds.$$

Intead of summing over time steps first, let us sum over states. This yields:
$$J(\pi) = \int_S \sum\limits_{t = 0}^\infty \gamma^t r^\pi(s) p(S_t=s|\rho_0,\pi) ds.$$

Let us write $\rho^\pi_{\rho_0}(s) = \sum\limits_{t = 0}^\infty \gamma^t p(S_t=s|\rho_0,\pi)$ for all $s \in S$. This quantity is called the *discounted state occupancy measure*. Although it is not a proper probability density (it does not sum to one), it is a proxy of how much time policy $\pi$ spends in state $s$ over the course of a trajectory, given that it started in $S_0$ drawn according to $\rho_0$.

So we have $J(\pi) = \langle \rho^\pi_{\rho_0}, r^\pi \rangle$. We can abuse the $\mathbb{E}$ notation and write:
$$J(\pi) = \mathbb{E}_{s\sim \rho^\pi_{\rho_0}}[r^\pi(s)] = \langle \rho^\pi_{\rho_0}, r^\pi \rangle.$$

So the value of policy $\pi$ is both a sum of rewards over time steps $\mathbb{E}[ \sum_{t = 0}^\infty \gamma^t R_t]$ and a sum of rewards across states \langle \rho^\pi_{\rho_0}, r^\pi \rangle, weighted by their occupancy measure under $(\pi,\rho_0)$.

Interestingly, searching for a policy which maximizes $J(\pi)$ boils down to searching for a policy's whose state occupancy measure is maximally aligned with the reward model.

<div class="alert alert-warning">

**Exercise**  
What does $\rho^\pi_{\rho_0}$ sum to? Normalize this measure to turn it into a probability distribution and correct the last expression of $J(\pi)$ above.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
The integral of $\rho^\pi_{\rho_0}$ across $S$ is:
\begin{align*}
\int_S \rho^\pi_{\rho_0}(s)ds &= \int_S \sum\limits_{t = 0}^\infty \gamma^t p(S_t=s|\rho_0,\pi) ds \\
 &= \sum\limits_{t = 0}^\infty \gamma^t \int_S  p(S_t=s|\rho_0,\pi) ds \\
 &= \sum\limits_{t = 0}^\infty \gamma^t \\
 &= \frac{1}{1-\gamma}
\end{align*}

So $\rho^\pi_{\rho_0}$ always sums to $\frac{1}{1-\gamma}$. A proper probability distribution would then be $(1-\gamma)\rho^\pi_{\rho_0}$ and a correct writing for $J(\pi)$ is thus:
$$J(\pi) = \mathbb{E}_{s\sim (1-\gamma)\rho^\pi_{\rho_0}}[r^\pi(s)]$$
</details>

<div class="alert alert-warning">

**Exercise**  
Can you derive the same result with a state-action occupancy measure, instead of a state-only occupancy measure?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
$$J(\pi) = \mathbb{E}_{(S_t,A_t)_{t\in\mathbb{N}}}\left[ \sum\limits_{t = 0}^\infty \gamma^t r(S_t,A_t) \right]$$

Let's define $\rho^\pi_{\rho_0}(s,a) = \sum\limits_{t = 0}^\infty \gamma^t p(S_t=s, A_t=a|\rho_0,\pi)$. Then, given a reward model $r(s,a))$:
$$J(\pi) = \langle \rho^\pi_{\rho_0}, r \rangle = \mathbb{E}_{s\sim (1-\gamma)\rho^\pi_{\rho_0}}[r(s,a)].$$
</details>

<div class="alert alert-warning">

**Exercise**  
Consider a finite state space MDP. So $\rho_0$ is a line vector and $p^\pi$ is a square matrix. In a previous exercise, we derived the expression of $p(S_t=s|\rho_0,\pi)$. Use this expression in $\rho^\pi_{\rho_0}(s)$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
We had $p(S_t=s|\rho_0,\pi) = \rho_0 (p^\pi)^t$.

So $\rho^\pi_{\rho_0}(s) = \sum\limits_{t = 0}^\infty \rho_0 (\gamma p^\pi)^t$
</details>

<div class="alert alert-warning">

**Exercise**  
Write a function that estimates $\rho^\pi_{\rho_0}$ to compute the value of the initial state in FrozenLake. Compare to the Monte Carlo estimate built in a previous exercise.
</div>

In [ ]:
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line in the cell below to load a correction (then you can execute this code).

In [ ]:
# %load solutions/no_solution_yet.py

## Homework: Gymnasium

In this notebook, we have been using a suite of environments called [Gymnasium](https://gymnasium.farama.org/), which provided us with the FrozenLake environment. Gymnasium is a maintained fork of OpenAI’s Gym library which has become the *de facto* standard in terms of Python API for reinforcement learning environments. As Gym had a history of low maintenance and poor documentation, all development of Gym has been moved to Gymnasium.

The Gymnasium API has been used without introduction of explanations: it is time to explore a little more how it works.

<div class="alert alert-warning">

**Exercise**  
Read three specific pages in Gymnasium's documentation:  
- The [front page](https://gymnasium.farama.org/)
- The introduction to Gymnasium's [basic usage](https://gymnasium.farama.org/content/basic_usage/)
- The general API to all Gymnasium's [environments](https://gymnasium.farama.org/api/env/)

Also take a minute to browse through the different families of environments provided.
</div>

## Interpreting gamma

We motivated the introduction of $\gamma$ with the argument that $\sum\limits_{t = 0}^\infty R_t$ might grow unbounded. The solution was to *discount* future rewards by $\gamma^t$.

<div class="alert alert-warning">

**Exercise**  
Suppose rewards are drawn from the $[0,1]$ interval. What is the upper bound on $\sum\limits_{t = 0}^\infty \gamma^t R_t$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

The upper bound is $\frac{1}{1-\gamma}$.
</details>

Consider an MDP $\langle S, A, p, r\rangle$ and a policy $\pi$. 
Let's now append one state to $S$, which we will write $s_{null}$. This defines a new state space $S'$. We will consider $S$ is ordered and $s_{null}$ is the last element of $S'$. 
The action space remains the same.  
$s_{null}$ will play the role of a zero-reward, absorbing state, reachable from any other state.

When in any state $s \in S$, we first check for a transition to $s_{null}$ with probability $1-\gamma$. Otherwise, we transition to another state $s'$ with probability $p^\pi(s'|s)$. 
So $s_{null}$ is an absorbing state, reachable in one step from any other state with probability $1-\gamma$. 
All transitions from $s_{null}$ loop back to itself with probability one, whichever the action taken. Consequently, the policy $\pi$ can be trivially extended to $S'$ as a policy $\pi'$ which takes the same action as $\pi$ in all $s\in S$ and takes any action in $s_{null}$.

<div class="alert alert-warning">

**Exercise**  
Let $p^\pi$ be the transition matrix of the first MDP under $\pi$. Write the transition matrix $p^{\pi'}$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
Let's write $\mathbf{(1-\gamma)_{S}}$ the column vector composed of $|S|$ elements, all equal to $1-\gamma$.  
Let's write $\mathbf{0_S}$ the column vector composed of $|S|$ elements, all equal to $0$.  
Then:
$$p^{\pi'} = \left[ \begin{array}{cc} \gamma p^\pi & \mathbf{(1-\gamma)_{S}} \\ \mathbf{0_S}^T & 1 \end{array} \right]$$
</details>

So, in plain words, $s_{null}$ is a *terminal* state, and $\gamma$ is a probability of *non-termination* in the second MDP.

An iteresting aspect is that $s_{null}$ is accessible from *any* state of $S$ with the same probability.

Let's call *length* of a trajectory the first time step at which $s_{null}$ is encountered. So it is the number of transitions leading to $s_{null}$ (note that this includes the last transition to $s_{null}$).

<div class="alert alert-warning">

**Exercise**  
What is the probability of a trajectory of length greater or equal to $h$ time steps?  
What is the probability of a trajectory of length exactly equal to $h$ time steps?  
How is this probability distribution called?  
Deduce the average trajectory length.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
Reaching $s_{null}$ after at least $h$ time steps, means not having reached it before, which has probability $(1-\gamma)^h$.
    
$$\mathbb{P}(\textrm{length}\geq h) = (1-\gamma)^h$$

Hence the probability of a trajectory of length $h$ is:
\begin{align*}
\mathbb{P}(\textrm{length}= h) &= \mathbb{P}(h+1 > \textrm{length}\geq h)\\
&= \mathbb{P}(\textrm{length}\geq h) - \mathbb{P}(\textrm{length}\geq h+1)\\
&= (1-\gamma)^h - (1-\gamma)^{h+1}\\
&= \gamma (1-\gamma)^h
\end{align*}
    
So the length of a trajectory follows a geometric distribution of parameter $\gamma$. The average trajectory length is hence $\frac{1}{1-\gamma}$. We will sometimes call this quantity (a bit abusively) the **horizon of MDP $\langle S,A,p,r \rangle$ under the $\gamma$-discounted criterion**.
</details>

<div class="alert alert-warning">

**Exercise**  
Can you relate the value of $\pi'$ in the second MDP under the total reward criterion to that of $\pi$ in the first MDP under the $\gamma$ discounted criterion?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
    
We shall write $V^{\pi'}$ the value of $\pi'$ under the total reward criterion, and $V^\pi_\gamma$ that of $\pi$ under the $\gamma$-discounted criterion, to avoid ambiguities.

Let's consider the first transition of a trajectory starting in $s$. Either this transition moves to $s_{null}$ with probability $1-\gamma$, or it moves to a state $s'\in S$, with probability $\gamma p^\pi(s'|s)$. The transition to $s_{null}$ provides zero reward, while the transition to $s'$ provides $r^\pi(s,s')$. From $s'$, the expected gain is $V^\pi(s')$. So:
$$V^{\pi'}(s) = (1-\gamma)\cdot 0 + \mathbb{E}_{s' \sim \gamma p^\pi} [r^\pi(s,s') + V^\pi(s')].$$
    
But $\mathbb{E}_{s' \sim \gamma p^\pi}[\cdot] = \gamma \mathbb{E}_{s' \sim p^\pi}[\cdot]$, so:
$$V^{\pi'}(s) = \gamma \mathbb{E}_{s' \sim p^\pi} \left[r^\pi(s,s') + V^\pi(s')\right].$$
    
We can unfold this for the following time step, ie. states reached from $s'$:
$$V^{\pi'}(s) = \gamma \mathbb{E}_{s' \sim \gamma p^\pi} \left[r^\pi(s,s') + \gamma \mathbb{E}_{s'' \sim (p^\pi)^2} \left[r^\pi(s',s'') + V^\pi(s'') \right] \right].$$

Which, in the limit, yields:
$$V^{\pi'}(s) = \gamma \mathbb{E}_{s_t \sim (p^\pi)^t} \left[ \sum_{t=0}^\infty \gamma^t r^\pi(s_t,s_{t+1}) \right].$$

But we find there the definition of $V^\pi_\gamma$:
$$V^{\pi'}(s) = \gamma V^\pi_\gamma(s).$$

</details>

Consequently, to a constant multiplicative factor, the value of a trajectory under a $\gamma$-discounted criterion, is actually the value of playing the same game under a total reward criterion, but with a probability of termination of $1-\gamma$ at each time step. In other words, using the $\gamma$-discounted criterion boils down to taking into account a probability $\gamma$ of "staying alive" at each time step in the current game.